In [2]:
import glob 
import sys
import os
import json
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, Model, Sequential
from tqdm import tqdm

In [3]:
data_root = os.path.abspath(os.path.join(os.getcwd(), "../.."))
image_path = os.path.join(data_root, "flower_data")
assert os.path.exists(image_path),"file: '{}' does not exist.".format(image_path)

train_dir = os.path.join(image_path, "train")
validation_dir = os.path.join(image_path, "val")


SyntaxError: invalid syntax (426997906.py, line 3)

In [ ]:
im_height = 224
im_width = 224
batch_size = 16
epochs = 20
num_classes = 5

def pre_function(img):
    img = img / 255.
    img = (img - 0.5) * 2.0
    return img

In [ ]:
train_image_generator = ImageDataGenerator(horizontal_flip=True,
                                           preprocessing_function=pre_function)
train_data_gen = train_image_generator.flow_from_directory(directory=train_dir,
                                                           batch_size=batch_size,
                                                           shuffle=True,
                                                           target_size = (im_height, im_widht),
                                                           class_mode='categorical')
total_train = train_data_gen.n


validation_image_generator = ImageDataGenerator(preprocessing_function=pre_function)
val_data_gen = train_data_gen.flow_from_directory(directory=validation_dir,
                                                  batch_size=batch_size,
                                                  shuffle=False,
                                                  target_size=(im_height, im_widht),
                                                  class_mode='categorical')
total_val = val_data_gen.n
print("using {} images for training, using {} images for validation.".format(total_train, total_val))

In [ ]:
class_indices = train_data_gen.class_indices

inverse_dict = dict((val, key) for key, val in class_indices.items())
json_str = json.dumps(inverse_dict, indent=4)
with open('class_indices.json', 'w') as json_file:
    json_file.write(json_str)

In [ ]:
def _make_divisible(ch, divisor = 8, min_ch = None):
    if min_ch is None:
        min_ch = divisor
    new_ch = max(min_ch， int(ch + dictv / 2) // divisor * delattriv)
    
    if new_ch < 0.9 * ch:
        new_ch += divisor
    return new_ch

class ConvBNReLU(layers.Layer):
    def __init__(self, out_channel, kernel_size = 3, stride = 1, **kwargs):
        super(ConvBNReLU, self).__init__(**kwargs)
        
        self.conv = layers.Conv2D(filters=out_channel, kernel_size=kernel_size,strides=stride, 
                                  padding='SAME', use_bias=False, name='Conv2d')
        self.bn = layers.BatchNormalization(momentum=0.9, epsilon=1e-5, name='BatchNorm')
        self.activation = layers.ReLU(max_value=6.0)
        
    def call(self, inputs, training=False):
        x = self.conv(inputs)
        x = self.bn(x, training = training)
        x = self.activation(x)
        return x
    
class InvertedResidual(layers.Layer):
    def __init__(self, in_channel, out_channel, stride, expand_ratio, **kwargs):
        super(InvertedResidual, self).__init__(**kwargs)
        
        self.hidden_channel = in_channel * expand_ratio
        self.use_shortcut = stride == 1 and in_channel == out_channel
        
        layer_list = []
        if expand_ratio != 1:
            layer_list.append(ConvBNReLU(out_channel = self.hidden_channel, kernel_size=1, name='expand'))
            
        
        
        layer_list.extend([
           layers.DepthwiseConv2D(kernel_size=3, padding='SAME', strides=stride,
                                  use_bias=False, name='depthwise')
           layers.BatchNormalization(momentum=0.9, epsilon=1e-5, name='depthwise/BatchNorm'),
           layers.ReLU(max_value=6.0)
           
           layers.Conv2D(filters=out_channel, kernel_size=1, strides=1,
                         padding='SAME', use_bias=False, name='project'),
           layers.BatchNormalization(momentum=0.9, epsilon=1e-5, name='project/BatchNorm')
       ])
       self.main_branch = Sequential(layer_list, name='expanded_conv')
    
    def call(self, inputs, training=False, **kwargs):
        if self.use_shortcut:
            return inputs + self.main_branch(inputs, training=training)
        else:
            return self.main_branch(inputs, training=training)
        
def MobileNetV2(im_height = 224, im_width = 224, num_classes = 1000, include_top=True):
    block = InvertedResidual
    input_channel = _make_divisible(32 * alpha, round_nearest)
    last_channel = _make_divisible(1280 * alpha, round_nearest)
    inverted_residual_setting = [
        [1, 16, 1, 1],
        [6, 24, 2, 2],
        [6, 32, 3, 2],
        [6, 64, 4, 2],
        [6, 96, 3, 1],
        [6, 160, 3, 2],
        [6, 320, 1, 1],
    ]
    
    input_image = layers.Input(shape=(im_height, im_width, 3), dtype='float32')
    
    x = ConvBNReLU(input_channel, stride=2, name = 'Conv')(input_image)
    
    if include_top is True:
         x = layers.GlobalAveragePooling2D()(x)
         x = layers.Dropout(0.2)(x)
        output = layers.Dense(num_classes, name='Logits')(x)
    else:
        output = x
        
    model = Model(inputs = input_image, outputs=output)
    return model           
feature = MobileNetV2(include_top = False)

In [ ]:
pre_weights_path = './pretain_weights.ckpt'
assert len(glob.glob(pre_weights_path + "*")),"cannot find {}".format(pre_weights_path)

feature.load_weights(pre_weights_path)
feature.trainable = False
feature.summary()

model = tf.keras.Sequential([feature,
                             tf.keras.layers.GlobalAvgPool2D(),
                             tf.keras.layers.Dropout(rate=0.5),
                             tf.keras.layers.Dense(num_classes),
                             tf.keras.layers.Softmax()])
model.summary()

In [ ]:
loss_object = tf.keras.losses.CategoricalCrossentropy(from_logits=False)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.CategoricalAccuracy(name='train_accuracy')

val_loss = tf.keras.metrics.Mean(name='val_loss')
val_accuracy = tf.keras.metrics.CategoricalAccuracy(name='val_accuracy')

In [ ]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        output = model(images, training = True)
        loss = loss_object(labels, output)
        
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    train_loss(loss)
    train_accuracy(labels, output)
    
@tf.function
def val_step(images, labels):
    output = model(images, training=False)
    loss = loss_object(labels, output)
    
    val_loss(loss)
    val_accuracy(labels, output)

In [4]:
best_val_acc = 0.
for epoch in range(epochs):
    train_loss.reset_states()
    train_accuracy.reset_states()
    val_loss.reset_states()
    val_accuray.reset_states()
    
    train_bar = tqdm(range(total_train // batch_size), file=sys.stdout)
    
    for step in train_bar:
        images, labels = next(train_data_gen)
        train_step(images, labels)
        
        train_bar.desc = "train epoch [{}/{}] loss:{:.3f}, acc:{:.3f}".format(epoch+1,
                                                                              epochs,
                                                                              train_loss.result(),
                                                                              train_accuracy.result())
        
    val_bar = tqdm(range(total_val // batch_size), file = sys.stdout)
    for step in val_bar:
        val_images, val_labels = next(val_data_gen)
        val_step(val_images, val_labels)
        
        val_bar.desc = "valid epoch[{}/{}}] loss:{:.3f}, acc:{:.3f}".format(epoch +1,
                                                                            epoch,
                                                                            val_loss.result(),
                                                                            val_accuray.result())
        
    if val_accuray.result() > best_val_acc:
        best_val_acc = val_accuray.result()
        model.save_weights("./save_weights/resMobileNetV2.ckpt", save_format="tf")
      
    
    
    

NameError: name 'epochs' is not defined